In [1]:

# Import Libraries
import datetime as dt
import json
import os


import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

import src.data_processing.MIMIC.test_functions as tests

# LOAD CONFIGURATION 
with open("src/data_processing/MIMIC/MIMIC_PROCESSING_DEFAULT_VARS.json", "r") as f:
    DEFAULT_CONFIG = json.load(f)
    f.close()

if not os.path.exists(DEFAULT_CONFIG["SAVE_FD"]):
    os.makedirs(DEFAULT_CONFIG["SAVE_FD"])

In [5]:
try:
    assert os.path.exists(DEFAULT_CONFIG["SAVE_FD"] + "admissions_intermediate.csv")
    assert os.path.exists(DEFAULT_CONFIG["SAVE_FD"] + "vitals_intermediate.csv")

except AssertionError as e:
    raise e

In [7]:
# Print Information
print("\n\n ======== PROCESSING OUTCOMES ======== \n\n")

# Load previously processed data
adm_proc = pd.read_csv(
    DEFAULT_CONFIG["SAVE_FD"] + "admissions_intermediate.csv",
    index_col=0, 
    header=0, 
    parse_dates=["intime", "outtime", "intime_next", "outtime_next", "deathtime"]
)
vit_proc = (
    pd.read_csv(
    DEFAULT_CONFIG["SAVE_FD"] + "vitals_intermediate.csv", 
    index_col=0, 
    header=0, 
    parse_dates=DEFAULT_CONFIG["VITALS_TIME_VARS"]
    )
    .reset_index(drop=False)
    .assign(sampled_time_to_end=lambda x: pd.to_timedelta(x["sampled_time_to_end"]))  # pd does not load timedelta automatically
)


# Check correct computation of admissions and vitals
tests.test_admissions_processed_correctly(adm_proc)
tests.test_vitals_processed_correctly(vit_proc, config_dic=DEFAULT_CONFIG)



 ======== PROCESSING OUTCOMES ======== 



Testing admissions processed correctly...

Testing outtime is after intime.
Test passed!

Testing next transfer information is consistent.
Test passed!

Testing admission times are before death (if exists).
Test passed!

Testing ids are unique for params ('subject_id', 'hadm_id', 'stay_id', 'transfer_id_next')
Test passed for variable  subject_id!
Test passed for variable  hadm_id!
Test passed for variable  stay_id!
Test passed for variable  transfer_id_next!

Testing ids are complete for params ('subject_id', 'stay_id', 'intime', 'outtime')
Test passed for variable subject_id!
Test passed for variable stay_id!
Test passed for variable intime!
Test passed for variable outtime!
Test passed!
Admissions correctly computed! Safe to go ahead.

Testing vitals were processed correctly and make sense.

Testing ids are complete for params ('subject_id', 'stay_id', 'sampled_time_to_end')
Test passed for variable subject_id!
Test passed for variable st

100%|████████████████████████████| 8364/8364 [00:25<00:00, 332.38it/s]


Test passed!

Testing resampling data is linear from min to max per patient.


100%|███████████████████████████| 8364/8364 [00:03<00:00, 2695.11it/s]

Test passed!
Vitals seem correctly processed!


In [8]:
# Load core info
transfers_core = pd.read_csv(
    DEFAULT_CONFIG["DATA_FD"] + "core/transfers.csv", 
    index_col=None, 
    header=0, 
    parse_dates=["intime", "outtime"]
)
admissions_core = pd.read_csv(
    DEFAULT_CONFIG["DATA_FD"] + "core/admissions.csv",
    index_col=None,
    header=0,
    parse_dates=["admittime", "dischtime", "deathtime", "edregtime", "edouttime"]
)



In [143]:

"""
Step 1: Subset the set of transfers/admissions_core to the already processed cohort.

We do this by merging. 
"""

# Define Id for merging. We separate deathtime as one database registers only date, while the other
# registers everyting (i.e. up to second)
tr_merge_ids = [
    col for 
    col in vit_proc.columns.tolist() if
    col in transfers_core.columns.tolist() and
    "death" not in col
]
hadm_merge_ids = [
    col for
    col in vit_proc.columns.tolist() if
    col in admissions_core.columns.tolist() and
    "death" not in col
]
merge_ids = ["subject_id", "hadm_id", "stay_id"]         # Useful simplication

# Inner merge for transfers core
transfers_S1 = (
    transfers_core
    .merge(
        vit_proc.drop_duplicates(subset=merge_ids),   # Drop duplicates as we don't need all the rows
        how="inner",
        on=tr_merge_ids
    )
    .dropna(subset=["hadm_id"])                 # Drop rows with no hadm_id as we can't compare with transfers
    .sort_values(by=merge_ids, ascending=True) # Sort by subject_id and stay_id
)

# Inner merge for admissions core
admissions_S1 = (
    admissions_core
    .merge(
        vit_proc.drop_duplicates(subset=merge_ids), # only want one obvs per admission for merging
        how="inner",
        on=hadm_merge_ids
    )
    .dropna(subset=["hadm_id"])            # Drop rows with no hadm_id as we can't compare with transfers
    .sort_values(by=merge_ids, ascending=True) # Sort by subject_id and stay_id
)

# Testing and save
tests.test_ids_subset_of_cohort(transfers_S1, vit_proc, *merge_ids)
tests.test_ids_subset_of_cohort(admissions_S1, vit_proc, *merge_ids)
tests.test_is_complete_ids(transfers_S1, *merge_ids, "stay_id")
tests.test_is_complete_ids(admissions_S1, *merge_ids, "stay_id")

# Check processing and correctdeness
transfers_S1.to_csv(DEFAULT_CONFIG["SAVE_FD"] + "transfers_S1.csv", header=True, index=True)



Testing ('subject_id', 'hadm_id', 'stay_id') are subset of cohort data.
Test passed!

Testing ('subject_id', 'hadm_id', 'stay_id') are subset of cohort data.
Test passed!

Testing ids are complete for params ('subject_id', 'hadm_id', 'stay_id', 'stay_id')
Test passed for variable subject_id!
Test passed for variable hadm_id!
Test passed for variable stay_id!
Test passed for variable stay_id!

Testing ids are complete for params ('subject_id', 'hadm_id', 'stay_id', 'stay_id')
Test passed for variable subject_id!
Test passed for variable hadm_id!
Test passed for variable stay_id!
Test passed for variable stay_id!


In [145]:
admissions_S1

,subject_id,hadm_id,admittime,dischtime,deathtime_x,admission_type,admission_location,discharge_location,insurance,language,...,transfer_id,eventtype,careunit,intime,outtime,gender,age,ESI,deathtime_y,charttime_ub
1869,10000084,23052089,2160-11-21 01:56:00,2160-11-25 14:52:00,NaT,EW EMER.,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Medicare,ENGLISH,...,35203156,ED,Emergency Department,2160-11-20 20:36:00,2160-11-21 03:20:00,1,72,2.0,NaT,2160-11-21 03:20:00
3294,10001176,23334588,2186-11-29 03:56:00,2186-12-02 15:00:00,NaT,EU OBSERVATION,EMERGENCY ROOM,NaN,Medicare,ENGLISH,...,39677610,ED,Emergency Department,2186-11-28 21:34:00,2186-11-29 05:01:00,0,64,3.0,NaT,2186-11-29 05:01:00
5413,10001180,21102262,2192-05-23 04:58:00,2192-05-23 16:43:00,NaT,EU OBSERVATION,EMERGENCY ROOM,NaN,Other,ENGLISH,...,33306350,ED,Emergency Department,2192-05-22 23:46:00,2192-05-23 05:27:00,0,33,3.0,NaT,2192-05-23 05:27:00
2565,10001217,24597018,2157-11-18 22:56:00,2157-11-25 18:00:00,NaT,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Other,?,...,39866888,ED,Emergency Department,2157-11-18 17:38:00,2157-11-19 01:24:00,0,55,3.0,NaT,2157-11-19 01:24:00
2857,10004719,21197153,2183-08-30 18:28:00,2183-09-03 12:34:00,NaT,OBSERVATION ADMIT,EMERGENCY ROOM,HOME,Medicare,ENGLISH,...,36819102,ED,Emergency Department,2183-08-30 14:38:00,2183-08-30 20:32:00,0,66,3.0,NaT,2183-08-30 20:32:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,19993951,23793933,2139-02-18 20:03:00,2139-02-19 15:49:00,NaT,EU OBSERVATION,EMERGENCY ROOM,NaN,Medicare,ENGLISH,...,33496052,ED,Emergency Department,2139-02-18 11:39:00,2139-02-18 20:03:54,1,72,2.0,NaT,2139-02-18 20:03:54
4740,19996783,25894657,2188-03-05 20:17:00,2188-03-14 17:15:00,NaT,DIRECT EMER.,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,?,...,39421649,ED,Emergency Department,2188-03-05 15:06:00,2188-03-05 21:40:00,1,89,3.0,NaT,2188-03-05 21:40:00
4901,19997473,27787494,2173-09-11 00:53:00,2173-10-02 15:50:00,NaT,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,...,39636284,ED,Emergency Department,2173-09-10 22:20:00,2173-09-11 01:54:00,0,82,2.0,NaT,2173-09-11 01:54:00
5547,19997576,25548363,2187-10-07 22:35:00,2187-10-10 19:50:00,NaT,OBSERVATION ADMIT,TRANSFER FROM HOSPITAL,HOME,Medicare,ENGLISH,...,39468668,ED,Emergency Department,2187-10-07 18:43:00,2187-10-07 23:42:00,0,80,2.0,NaT,2187-10-07 23:42:00
